In [2]:
from sympy import sift, symbols, init_printing, parse_expr, Add, Symbol
init_printing(use_unicode=False, wrap_line=False)

In [21]:
class CrnWriter:
    def __init__(self, expr, dependent_variable):
        self.dv = dependent_variable
        self.dv1 = Symbol(f"{dependent_variable}_1")
        self.dv2 = Symbol(f"{dependent_variable}_2")
        self.dv1p = Symbol(f"{dependent_variable}_1^{{'\''}}")
        self.dv2p = Symbol(f"{dependent_variable}_2^{{'\''}}")
        self.p_plus = Symbol("p^{+}")
        self.p_minus = Symbol("p^{-}")
        self.p_polynomial = self.gen_p_polynomial()
        self.expr = parse_expr(expr)
        self.raw_parts = self.split_expr()
        self.pos_terms = self.raw_parts[0]
        self.neg_terms = self.raw_parts[1]
        self.pos = self.rewrite_pos()
        self.neg = self.rewrite_neg()
        
    def gen_p_polynomial(self):
        dv = self.dv[:1]
        dv1 = Symbol(f"{dv}_1")
        dv2 = Symbol(f"{dv}_2")
        p_polynomial = (self.p_plus+self.p_minus)*dv1*dv2
        return p_polynomial

    def split_expr(self):
        ispos = lambda x: x.as_coeff_Mul()[0].is_positive
        pos, neg = sift(Add.make_args(self.expr), ispos, binary=True)
        return pos, neg
    
    def rewrite_pos(self):
        pos_expr = ""
        for term in self.pos_terms:
            pos_expr += str(term) + "+" 
        pos_expr = pos_expr[:-1]
        return parse_expr(pos_expr)

    def rewrite_neg(self):
        neg_expr = ""
        for term in self.neg_terms:
            neg_expr += str(term)
        neg_expr = neg_expr[1:]
        return neg_expr

    def __str__(self):
        return f"""CrnWriter
                raw_parts: {self.raw_parts}
                positive_terms: {self.pos_terms}
                negative_terms: {self.neg_terms}
                pos_expr: {self.pos}-{self.p_polynomial}
                neg_expr: {self.neg}-{self.p_polynomial}
                """

    def dict(self):
        return vars(self)
    


In [23]:
w_prime = "-(w-1)"
writer = CrnWriter(w_prime, "w_prime")
print(writer)

CrnWriter
                raw_parts: ([1], [-w])
                positive_terms: [1]
                negative_terms: [-w]
                pos_expr: 1-w_1*w_2*(p^{+} + p^{-})
                neg_expr: w-w_1*w_2*(p^{+} + p^{-})
                
